# How to use chat models to call tools
参考: [How to use chat models to call tools](https://python.langchain.com/docs/how_to/tool_calling/)
- Chat models
- Tool calling
- Tools
- Output parsers
- 请记住，“工具调用”这个名字让人以为模型会直接执行某些操作，但实际上并非如此！
- **模型只会生成工具的参数，而运行工具（或不运行）则由用户决定。**
- 工具调用是一种**生成模型结构化输出**的一般技术，即使你无意调用任何工具，你也可以使用它。例如，**从非结构化文本中提取信息**就是一种使用场景。
    - 工具调用不是所有的模型都支持的,支持tool的模型会更方便调用tool,见: [LLM providers](https://python.langchain.com/docs/integrations/chat/)
    - 不支持tool的模型可以通过ChatOpenAI类设置base_url,返回的llm 支持结构化输出和bind_tools


## chat models

In [1]:
import sys
sys.path.append('/Users/ericyoung/ysx/code/github-study/langChain-rookie')
from env_utils import load_environment_variables

load_environment_variables()

__file__: /Users/ericyoung/ysx/code/github-study/langChain-rookie/env_utils.py
dotenv_path1: /Users/ericyoung/ysx/code/github-study/langChain-rookie/.env
load env ok


### 支持tool的 chat model

In [2]:
# Ollama
from langchain_ollama import ChatOllama
# llm = ChatOllama(
#     model="qwen2.5:latest", # gemma3:latest(不支持tool)
#     temperature=0.7,
#     # other params...
# )

### 不支持tool的chat model

In [3]:
# 任何chat model
from models import MyOpenAIModel

model = MyOpenAIModel()
response = model.generate("你好，介绍一下你自己")
print(response)

您好！我是Gemma，一个由 Google DeepMind 训练的大型语言模型。我是一个开放权重的模型，这意味着我的权重是公开的，您可以自由地使用和修改我。

我可以理解和生成文本，可以帮助您进行各种任务，例如：

*   回答问题
*   创作不同类型的文本格式（诗歌、代码、剧本、音乐作品、电子邮件、信件等）
*   翻译语言
*   总结文本

我是由 Gemma 团队构建的，并且一直在不断学习和改进。

您想和我聊些什么呢？ 😊您好！我是Gemma，一个由 Google DeepMind 训练的大型语言模型。我是一个开放权重的模型，这意味着我的权重是公开的，您可以自由地使用和修改我。

我可以理解和生成文本，可以帮助您进行各种任务，例如：

*   回答问题
*   创作不同类型的文本格式（诗歌、代码、剧本、音乐作品、电子邮件、信件等）
*   翻译语言
*   总结文本

我是由 Gemma 团队构建的，并且一直在不断学习和改进。

您想和我聊些什么呢？ 😊


### ⭐️ 不支持tool模型 利用 ChatOpenAI (使用base_url) 实现 bind_tools
- 支持结构化输出, with_structured_output方法
- 支持使用工具, bind_tools方法
- llm = ChatOpenAI(
                model="gpt-4o",
                temperature=0,
                max_tokens=None,
                timeout=None,
                max_retries=2,
                # api_key="...",
                # base_url="...",
                # organization="...",
                # other params...
            )

In [4]:
from models import get_base_url_model_with_tools
llm = get_base_url_model_with_tools()
print(llm.invoke("你是谁?").content)

我是Gemma，一个由 Google DeepMind 训练的大型语言模型。我是一个开放权重的模型，可以广泛地被公众使用。

你可以把我当作一个文本生成工具来理解。我可以根据你给我的提示生成文本、翻译语言、回答问题等等。

请注意，我没有连接到互联网，也无法访问实时信息。


## ?定义工具
Defining tool schemas  定义工具模式
- 为了使模型能够调用工具，我们需要传递描述工具功能及其参数的工具模式。
- 支持工具调用功能的聊天模型实现了一个**.bind_tools() 方法**，用于向模型传递工具模式。
- 定义工具模式的方式主要有:
    - Python 函数（带有类型提示和文档字符串）
    - Pydantic 模型、
    - TypedDict 类或
    - LangChain Tool 对象传递。--- 通过@tool装饰器实现,详见: [How to create tools](https://python.langchain.com/docs/how_to/custom_tools/#creating-tools-from-functions)
- 模型后续的调用将与提示一起传递这些工具模式。

### Python 函数

In [5]:
# The function name, type hints, and docstring are all part of the tool
# schema that's passed to the model. Defining good, descriptive schemas
# is an extension of prompt engineering and is an important part of
# getting models to perform well.
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b


def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

### ?LangChain 工具
-  通过@tool装饰器实现,详见: [How to create tools](https://python.langchain.com/docs/how_to/custom_tools/#creating-tools-from-functions)

### Pydantic 类
- 使用 Pydantic 定义不带伴随函数的模式
    - 除非提供默认值，否则所有字段都是 required

In [6]:
from pydantic import BaseModel, Field


class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

### TypedDict类-带类型提示
- 需要 langchain-core>=0.2.25
- 可使用 TypedDicts 和注解

In [7]:
from typing_extensions import Annotated, TypedDict


class add(TypedDict):
    """Add two integers."""

    # Annotations must have the type and can optionally include a default value and description (in that order).
    a: Annotated[int, ..., "First integer"]
    b: Annotated[int, ..., "Second integer"]


class multiply(TypedDict):
    """Multiply two integers."""

    a: Annotated[int, ..., "First integer"]
    b: Annotated[int, ..., "Second integer"]


tools = [add, multiply]

## bind_tools
- 将这些模式绑定到聊天模型，我们将使用 .bind_tools() 方法。
- 此方法处理将 add 和 multiply 模式转换为模型所需的正确格式。
- 每次调用模型时，工具模式将被传递给它。
- 可以看到响应中的tool_calls中使用了multiply工具方法,工具调用成功

In [8]:
llm_with_tools = llm.bind_tools(tools)

query = "What is 3 * 12?"

llm_with_tools.invoke(query)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '576632143', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 563, 'total_tokens': 600, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gemma-3-4b-it', 'system_fingerprint': 'gemma-3-4b-it', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0075b18f-9a68-4116-ac31-6893053f8b09-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': '576632143', 'type': 'tool_call'}], usage_metadata={'input_tokens': 563, 'output_tokens': 37, 'total_tokens': 600, 'input_token_details': {}, 'output_token_details': {}})

## 工具调用 Tool calls
- LLM响应中 .tool_calls 属性就是工具调用 ToolCall对象 列表, 请注意聊天模型可以一次调用多个工具。
- ToolCall 是一个带类型的字典，包含工具名称name、参数值字典args以及（可选地）一个标识符id。没有工具调用的消息默认将此属性设置为空列表。
- .tool_calls 属性应包含有效的工具调用。
    - 请注意，有时模型提供者可能会输出格式错误的工具调用（例如，无效的 JSON 参数）。
    - 在这种情况下，当解析失败时， .invalid_tool_calls 属性中会填充 InvalidToolCall 实例。
    - InvalidToolCall 可以具有名称、字符串参数、标识符和错误消息。

In [9]:
query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': '541637767',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 11, 'b': 49},
  'id': '646266744',
  'type': 'tool_call'}]

In [10]:
llm_with_tools.invoke(query)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '416249349', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'multiply'}, 'type': 'function'}, {'id': '802804964', 'function': {'arguments': '{"a":11,"b":49}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 575, 'total_tokens': 649, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gemma-3-4b-it', 'system_fingerprint': 'gemma-3-4b-it', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-762062ec-3caa-4a38-924a-d23cfbc9c6b6-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': '416249349', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': '802804964', 'type': 'tool_call'}], usage_metadata={'input_tokens': 575, 'output_tokens': 74, 'total_tokens': 649, 'input_token_details': {}, 'output_token_details': {}})

## Parsing  解析
- 如果需要，可以进一步处理输出。例如，我们可以使用 PydanticToolsParser 将 .tool_calls 上已填充的现有值转换为 Pydantic 对象：

In [11]:
from langchain_core.output_parsers import PydanticToolsParser
from pydantic import BaseModel, Field


class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


chain = llm_with_tools | PydanticToolsParser(tools=[add, multiply])
chain.invoke(query)

[multiply(a=3, b=12), add(a=11, b=49)]

## ?如何将工具输出传递给聊天模型
[How to pass tool outputs to chat models](https://python.langchain.com/docs/how_to/tool_results_pass_to_model/)

In [19]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

llm_with_tools = llm.bind_tools(tools)

### 让模型调用一个工具。我们将把它添加到一个消息列表中，我们将把这些消息视为对话历史：

In [20]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': '783362832', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': '863709297', 'type': 'tool_call'}]


### 如果我们调用一个 LangChain Tool 并传入一个 ToolCall ，我们将会自动获得一个 ToolMessage 可以反馈给模型

- 该功能在 langchain-core == 0.2.19 中添加。请确保您的包是最新的。
- 如果您使用的是 langchain-core 的较早版本，请需要从工具中提取 args 字段并手动构建一个 ToolMessage

In [ ]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

### 最后，我们将使用工具结果调用模型。模型将利用这些信息生成对我们原始查询的最终答案：

In [23]:
llm_with_tools.invoke(messages)

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 0, 'prompt_tokens': 670, 'total_tokens': 670, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gemma-3-4b-it', 'system_fingerprint': 'gemma-3-4b-it', 'finish_reason': 'stop', 'logprobs': None}, id='run-1683833f-8551-4bcc-ad3d-c2d10d19fb1a-0', usage_metadata={'input_tokens': 670, 'output_tokens': 0, 'total_tokens': 670, 'input_token_details': {}, 'output_token_details': {}})

## ?如何将运行时值传递给工具
[How to pass run time values to tools](https://python.langchain.com/docs/how_to/tool_runtime/)